In [5]:
# 导入所需的库
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

# 定义一个函数来计算点云的表面变化因子
def surface_variation_factor(points, k):
    # 将点云数据转换为 numpy 数组
    points = np.asarray(points)
    # 计算每个点的最近邻点
    kdtree = o3d.geometry.KDTreeFlann(points)
    neighbors = []
    for i in range(len(points)):
        # 检查点是否包含 NaN 或无穷大的值
        if np.any(np.isnan(points[i])) or np.any(np.isinf(points[i])):
            print(f"Point {i} contains NaN or infinite values!")
            continue
        # 确保 k 不大于点的数量
        k = min(k, len(points))
        try:
            _, idx, _ = kdtree.search_knn_vector_3d(points[i], k)
            neighbors.append(idx)
        except RuntimeError as e:
            print(e)
            neighbors.append(np.arange(len(points)))
    # 计算每个点的法向量
    normals = np.zeros_like(points)
    for i in range(len(points)):
        p = points[i]
        n = points[neighbors[i]]
        # 使用 PCA 方法求解法向量
        cov = np.cov(n.T)
        w, v = np.linalg.eig(cov)
        normal = v[:, np.argmin(w)]
        # 调整法向量的方向
        if np.dot(p, normal) > 0:
            normal = -normal
        normals[i] = normal
    # 计算每个点的表面变化因子
    svf = np.zeros(len(points))
    for i in range(len(points)):
        p = points[i]
        n = normals[i]
        m = points[neighbors[i]]
        # 使用公式计算表面变化因子
        svf[i] = np.sum(np.abs(np.dot(m - p, n))) / k
    # 返回法向量和表面变化因子
    return normals, svf

# 定义一个函数来根据表面变化因子对点云进行分类
def classify_points(points, svf, threshold):
    # 将点云数据转换为 numpy 数组
    points = np.asarray(points)
    # 根据阈值将点云分为平滑区域和非平滑区域
    smooth = points[svf < threshold]
    non_smooth = points[svf >= threshold]
    # 返回分类后的点云
    return smooth, non_smooth

# 定义一个函数来对点云进行滤波
def filter_points(points, k, threshold):
    # 计算点云的表面变化因子
    normals, svf = surface_variation_factor(points, k)
    # 根据表面变化因子对点云进行分类
    smooth, non_smooth = classify_points(points, svf, threshold)
    # 对平滑区域的点云进行均值滤波
    smooth_filtered = np.zeros_like(smooth)
    kdtree = o3d.geometry.KDTreeFlann(smooth)
    for i in range(len(smooth)):
        _, idx, _ = kdtree.search_knn_vector_3d(smooth[i], k)
        smooth_filtered[i] = np.mean(smooth[idx], axis=0)
    # 对非平滑区域的点云进行双边滤波
    non_smooth_filtered = np.zeros_like(non_smooth)
    kdtree = o3d.geometry.KDTreeFlann(non_smooth)
    for i in range(len(non_smooth)):
        _, idx, dist = kdtree.search_knn_vector_3d(non_smooth[i], k)
        # 使用高斯函数计算空间权重和光度权重
        sigma_s = np.std(dist)
        sigma_r = np.std(non_smooth[idx] - non_smooth[i])
        w_s = np.exp(-dist**2 / (2 * sigma_s**2))
        w_r = np.exp(-np.sum((non_smooth[idx] - non_smooth[i])**2, axis=1) / (2 * sigma_r**2))
        w = w_s * w_r
        # 使用加权平均计算滤波后的点
        non_smooth_filtered[i] = np.sum(w[:, np.newaxis] * non_smooth[idx], axis=0) / np.sum(w)
    # 合并滤波后的点云
    filtered = np.vstack((smooth_filtered, non_smooth_filtered))
    # 返回滤波后的点云
    return filtered

# 读取点云数据
pcd = o3d.io.read_point_cloud("bunny.pcd")
# 设置参数
k = 20 # 最近邻点的个数
threshold = 0.1 # 表面变化因子的阈值
# 对点云进行滤波
filtered = filter_points(pcd.points, k, threshold)
# 创建一个新的点云对象
pcd_filtered = o3d.geometry.PointCloud()
pcd_filtered.points = o3d.utility.Vector3dVector(filtered)
# 可视化原始点云和滤波后的点云
o3d.visualization.draw_geometries([pcd, pcd_filtered])


search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn_vector_3d() error!
search_knn

KeyboardInterrupt: 